# A. Librerias y datos

In [14]:
import os
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

# B. Funciones

In [15]:
def set_google_maps_url(lat, lng):
    return f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"

def set_street_view_url(lat, lng):
    return f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"

# C. Prueba

In [16]:
url = 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas'

response = requests.get(url)
print(response.status_code)

200


In [17]:
if response.status_code == 200:
    html = response.text
    with open('./results/nexoinmobiliario.html', 'w') as f:
        f.write(html)
        
    print(f"Archivo guardado en ./results/nexoinmobiliario.html")
else:
    print(f"Error: {response.status_code}")


Archivo guardado en ./results/nexoinmobiliario.html


# D. Automatizar

In [24]:
# Listar todos los proyectos

# A. Leer el archivo html
with open('./results/nexoinmobiliario.html', 'r', encoding='utf-8', errors='ignore') as infile:
        lines = infile.readlines()

# B. Buscar la línea que contiene la palabra 'url'
cadena = ''
for i in range(len(lines) - 1, 0, -1):
    if '"url":' in lines[i]:
        cadena = lines[i]
        break

# C. Buscar las posiciones de 'url'
posiciones_url = []
posicion = cadena.find('"url":')

while posicion != -1:
    posiciones_url.append(posicion)
    posicion = cadena.find('"url":', posicion + 1)

# D. Buscar las posiciones de 'project_id'
posiciones_ids = []
posicion = cadena.find('"project_id":')

while posicion != -1:
    posiciones_ids.append(posicion)
    posicion = cadena.find('"project_id":', posicion + 1)

# E. Extraer los links
urls_list = []
for a, b in zip(posiciones_url, posiciones_ids):
    url = ''.join(list(cadena)[a + 7:b - 2])
    url = url.replace('\x5C', '')  
    url = str(url)

    base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
    urls_list.append(base)

len(urls_list)


739

In [25]:
# Extraer info de cada proyecto
urls_list = urls_list[:4]
urls_list

['https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2229-park-san-miguel',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3074-castillo-de-mar-condominio-club-de-playa',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2075-hope',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3538-libert']

In [26]:
# 4. Extraer los datos de cada proyecto
i = 0
modelos_list = []
dataframes_list = []
for url in urls_list:
    # A. Obtener la URL por request
    response = requests.get(url)
    # print(response.status_code,':' ,url)

    if response.status_code == 200:
        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        #### A. Datos generales
        # Extrae el texto de <h1>Eres Fase 2</h1>
        try:
            proyecto = soup.find('h1').get_text(strip=True)
        except:
            proyecto = '-'
        # print(f'Proyecto: {proyecto}')

        # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
        try:
            direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
        except:
            direccion = '-'
        # print(f'Dirección: {direccion}')

        # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
        try:
            distrito_nexo = soup.find('p', class_='Project-header-address').find_next_sibling('p').get_text(strip=True).replace('-', '').strip()
        except:
            distrito_nexo = '-'
        # print(f'Distrito: {distrito}')

        # Extrae el precio desde <p class="Project-header-price"><span class="title-price">Precio desde</span> <strong> S/. 319,118</strong></p>
        try:
            precio_desde = soup.find('p', class_='Project-header-price').strong.get_text(strip=True)
        except:
            precio_desde = '-'
        # print(f'Precio desde: {precio_desde}')

        # Extrae el valor de referencia <p class="title-price">(<span>valor referencia </span><strong>$ 84,940</strong>)</p>
        try:
            valor_referencia = soup.find('p', class_='title-price').strong.get_text(strip=True)
        except:
            valor_referencia = '-'
        # print(f'Valor de referencia: {valor_referencia}')

        # Listar las áreas comunes
        try:
            areas_lista = soup.find('ul', class_='Project-areas-list')
            elementos_lista = areas_lista.find_all('li')
            areas_comunes = ', '.join(elemento.get_text(strip=True) for elemento in elementos_lista)
        except:
            areas_comunes = '-'

        # Encuentra el campo de entrada para la latitud
        try:
            input_latitud = soup.find('input', id='latitude')
            latitud = input_latitud['value']
        except:
            latitud = '-'
        # print(f'Latitud: {latitud}')

        try:
            input_longitud = soup.find('input', id='longitude')
            longitud = input_longitud['value']
        except:
            longitud = '-'
        # print(f'Longitud: {longitud}')

        try:
            url_google_maps = set_google_maps_url(latitud, longitud)
        except:
            url_google_maps = '-'


        # print('')
        # print('Datos de la inmobiliaria')
        #### B. Datos de la inmobiliaria
        try:
            # Encuentra el div con la clase "bx-data-project box-st"
            div_informacion_proyecto = soup.find('div', class_='bx-data-project box-st')

            if div_informacion_proyecto:
                # Encuentra todas las filas de la tabla dentro del div
                filas_tabla = div_informacion_proyecto.find('table').find_all('tr')

                # Itera sobre las filas de la tabla e imprime la información
                for fila in filas_tabla:
                    columnas = fila.find_all('td')
                    if len(columnas) == 2:
                        try:
                            etiqueta = columnas[0].strong.get_text(strip=True)
                            valor = columnas[1].get_text(strip=True)
                            
                            if etiqueta == 'Tipo de inmueble':
                                tipo_inmueble = valor
                            elif etiqueta == 'Área total':
                                area_total = valor
                            elif etiqueta == 'Dormitorios':
                                dormitorios = valor
                            elif etiqueta == 'Etapa del proyecto':
                                etapa_proyecto = valor
                            elif etiqueta == 'Fecha de entrega':
                                fecha_entrega = valor
                            elif etiqueta == 'Financiamiento':
                                financiamiento = valor
                            elif etiqueta == 'Inmobiliaria Nombre':
                                inmobiliaria_nombre = valor
                            
                            # print(f'{etiqueta} -> {valor}')

                        except:
                            pass

                # También puedes extraer información específica usando el método find para elementos específicos
                # Por ejemplo, para obtener la información de la inmobiliaria
                inmobiliaria_nombre = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('h2').get_text(strip=True)
                # print(f'Inmobiliaria Nombre: {inmobiliaria_nombre}')
        except:
            tipo_inmueble = '-'
            area_total = '-'
            dormitorios = '-'
            etapa_proyecto = '-'
            fecha_entrega = '-'
            financiamiento = '-'
            inmobiliaria_nombre = '-'
            # print('No se encontró información de la inmobiliaria')

        #### C. Modelos disponibles
        try:
            modelos_disponibles = soup.find_all('div', class_='Project-available-model')
            modelos_list.append([  proyecto,modelos_disponibles, distrito_nexo, tipo_inmueble
                                 , etapa_proyecto, fecha_entrega, financiamiento
                                 , inmobiliaria_nombre, areas_comunes
                                 , url_google_maps, url])
            # print('')
            # print('Modelos disponibles')
            # print(f'Cantidad de modelos: {len(modelos_disponibles)}')
        except:
            pass

        #### D. Unir los datos en un diccionario
        proyecto_data = {
            'Proyecto': proyecto,
            'Dirección': direccion,
            'Distrito Nexoinmobiliario': distrito_nexo,
            'Precio Desde': precio_desde,
            'Valor de Referencia': valor_referencia,
            'Latitud': latitud,
            'Longitud': longitud,
            'Tipo de Inmueble': tipo_inmueble,
            'Área Total': area_total,
            'Dormitorios': dormitorios,
            'Etapa del Proyecto': etapa_proyecto,
            'Fecha de Entrega': fecha_entrega,
            'Financiamiento': financiamiento,
            'Inmobiliaria Nombre': inmobiliaria_nombre,
            'Áreas Comunes': areas_comunes,
            'URL Nexoinmobiliario': url,
            'URL Google Maps': url_google_maps

        }

        # Convierte el diccionario en un DataFrame
        df = pd.DataFrame([proyecto_data])
        dataframes_list.append(df)

        # Incrementa el contador
        print(i, '-> ', proyecto , '-> ', url)
        i += 1
        
        # Tiempo de espera
        time.sleep(1.21)

0 ->  Park San Miguel - Departamento en San Miguel ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2229-park-san-miguel
1 ->  CASTILLO DE MAR - CONDOMINIO CLUB DE PLAYA - Lote en Culebras ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3074-castillo-de-mar-condominio-club-de-playa
2 ->  Hope - Departamento en La Victoria ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2075-hope
3 ->  LIBERTÀ - Departamento en Pueblo Libre ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3538-libert


In [29]:
# 1. Concatenar los DataFrames
df_proyectos = pd.concat(dataframes_list)

# 2. Eliminar las columnas de Latitud y Longitud
df_proyectos = df_proyectos.drop(columns=['Latitud', 'Longitud'])
df_proyectos.head(3)

,Proyecto,Dirección,Distrito Nexoinmobiliario,Precio Desde,Valor de Referencia,Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,URL Google Maps
0,Park San Miguel - Departamento en San Miguel,Jr. Gran Mariscal Ramon Castilla 740,San Miguel,-,"$ 116,043",Departamento,62.98 a 106.81 m2,3 a 3,Entrega inmediata,"31 de Enero, 2024",Financiamiento Propio,GDC INMOBILIARIA,"Gimnasio, Jardin interior, Lobby, Parque priva...",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps/search/?api=1&quer...
0,CASTILLO DE MAR - CONDOMINIO CLUB DE PLAYA - L...,KM 314 PANAMERICANA NORTE,Culebras,-,"S/. 111,049",Lote,200.00 a 200.00 m2,3 a 3,En construcción,"01 de Febrero, 2024",Financiamiento Propio,GRV CORPORATIVA,"Área de juegos, Áreas verdes, Cancha de Frontó...",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps/search/?api=1&quer...
0,Hope - Departamento en La Victoria,A.V Canada 1075,Santa Catalina La Victoria,-,"$ 93,969",Departamento,53.67 a 71.23 m2,2 a 3,Entrega inmediata,"29 de Febrero, 2024",Banco BCP,URBANA PERÚ,"Gimnasio, Lobby, Sala de Niños, Sala de usos M...",https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps/search/?api=1&quer...
